In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import re
from time import sleep
import unicodedata
import os
from IPython.display import display, Markdown

pd.set_option('display.max_colwidth', -1)

s = requests.Session()

senha = open(u'\\\\172.20.0.45\\jornalismo novo\_Utilit\xe1rios\password.txt').read()

proxy = ({
    'http':'http://rogerio.moreira:{}@172.20.0.75:8080'.format(senha),
	'https':'https://rogerio.moreira:{}@172.20.0.75:8080'.format(senha)
	})

def clip(df, **kwargs):
	return df.to_clipboard(float_format=CommaFloatFormatter(), sep='\t', **kwargs)

class CommaFloatFormatter:
	def __mod__(self, x):
		return str(x).replace('.',',')

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.lower().decode('utf8')

def make_soup(url):
    html = s.get(url, proxies=proxy, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36'})
    soup = BeautifulSoup(html.content, "html5lib")
    return soup

def l_(list_):
    return ', '.join([str(el) for el in list_])

os.chdir(r'C:\Users\rogerio.moreira\Desktop')

# Conexão Google Sheets
import pygsheets

def connect():
    gc = pygsheets.authorize(outh_file=r"C:\Users\rogerio.moreira\Google Drive\Scripts\client_secret.json")
    id = '18sUrpOqWRRhIBAPSd-7d_N4YIrZtg2JU0nvTUwaBMtA'
    sh = gc.open_by_key(id)
    return sh

# Classes

class Partida():

    def acresc(self, index, dados):
        return int(dados[12][index].split(':')[-1])
    
    def res(self, lista):
        if lista[0]>lista[1]: return 'vitoria'
        elif lista[0]<lista[1]: return 'derrota'
        elif lista[0]==lista[1]: return 'empate'
        
   
    """
    Uma classe para registrar as partidas. Preciso de uma função para poder extrair os valores.
    Aqui, de início, ela pega dados universais da partida, tais como:
    - Número do jogo
    - Times participando (formato Mandante ? x ? Visitante)
    - Vencedor
    
    A ser implementado:
    - Horário da partida
    - Data da partida
    """
    def __init__(self, index):
        global dados
        base = 'http://45.4.112.22/SISGOL/WDER0700_Sumula.asp?SelStart1=2019&SelStop1=2019&SelStart2=292&SelStop2=292&SelStart3={}&Index=1&RunReport=Run+Report'.format
        soup = make_soup(base(index))

        if soup.table.tbody:
            dados = [[re.sub(' +$','', td.text) for td in el.findAll('td')] for el in soup.findAll('table','ReportTable')]
            tempo_1turno = 45 + self.acresc(3, dados)
            tempo_2turno = 45 + self.acresc(-1, dados)


            # Número do jogo
            self.idjogo = soup.find('strong', text=re.compile('Jogo N°: \d+')).text.split(' ')[-1]
            self.arbitro = [el for el in dados if 'Árbitro:' in el][0][1]

            # Times participante
            times_raw, placar_raw = [[el[1].split(' x '), 
                                      [int(num) for num in el[3].split(' x ')]] for el in dados if 'Resultado Final:' in el][0]

            self.mandante = {'equipe': times_raw[0], 'gols': placar_raw[0], 'resultado':self.res(placar_raw)}
            self.visitante = {'equipe': times_raw[1], 'gols': placar_raw[1], 'resultado':self.res(placar_raw[::-1])}


            # gols
            global gols_raw
            self.gols = []

            in_ = dados.index(['5.0 - GOLS'])
            out = dados.index(['6.0 - CARTÕES AMARELOS'])

            gols = dados[in_+1:out]
            gols_raw = gols

            for gol in gols[1:]:
                condicao = gol[3]
                jogador = gol[4]
                equipe = gol[5]
                if gol[1] == '1T': tempo = int(gol[0][:-1])
                else: tempo = int(gol[0][:-1]) + tempo_1turno
                self.gols.append({'jogador':jogador, 'tempo':tempo, 'equipe':equipe, 'condicao':condicao})


            # cartoes amarelos
            self.cartoesAmarelos = []

            in_ = dados.index(['6.0 - CARTÕES AMARELOS'])
            out = dados.index(['7.0 - CARTÕES VERMELHOS (2CA = Dupla Advertência)'],)

            cartoes = dados[in_+1:out]

            for cartao in cartoes[1:]:
                jogador = cartao[-3]
                if cartao[1] == '1T': tempo = int(cartao[0][:-1])
                else: tempo = int(cartao[0][:-1]) + tempo_1turno
                self.cartoesAmarelos.append({'jogador':jogador, 'tempo':tempo})


            # cartoes vermelhos
            self.cartoesVermelhos = []

            in_ = dados.index(['7.0 - CARTÕES VERMELHOS (2CA = Dupla Advertência)'],)
            out = dados.index(['8.0 - EXPULSÃO DE MEMBROS DA COMISSÃO TÉCNICA'],)

            cartoes = dados[in_+1:out]

            for cartao in cartoes[1:]:
                jogador = cartao[-3]
                if cartao[1] == '1T': tempo = int(cartao[0][:-1])
                else: tempo = int(cartao[0][:-1]) + tempo_1turno
                self.cartoesVermelhos.append({'jogador':jogador, 'tempo':tempo})


            # Substituições
            self.substitutos = []

            in_ = dados.index(['12.0 - SUBSTITUIÇÕES'])
            out = dados.index(['**1T = 1° Tempo | 2T = 2° Tempo | INT = Intervalo'])

            substitutos_raw = dados[in_+2:out]

            for el in substitutos_raw:
                equipe = el[2]

                if el[0] == '-':
                    tempoEvento = tempo_1turno
                else:
                    tempoEvento = int(el[0][:-1]) if el[1] == '1 TEMPO' else tempo_1turno+int(el[0][:-1])

                # Jogador que entrou
                jogador = el[3].split(' - ')[-1]
                tempoJogado = tempo_1turno + tempo_2turno - tempoEvento

                self.substitutos.append({'jogador':jogador, 
                                         'tempoEntrou':tempoEvento, 
                                         'tempoSaiu': tempo_1turno + tempo_2turno, 
                                         'tempoJogado':tempoJogado, 
                                         'situacao':'entrou',
                                         'equipe':equipe,
                                         'substituido':'',
                                         'substituiu':el[4].split(' - ')[-1]})

                # Jogador que saiu
                jogador = el[4].split(' - ')[-1]
                tempoJogado = tempoEvento

                if not jogador in [el['jogador'] for el in self.substitutos]:
                    self.substitutos.append({'jogador':jogador, 
                                             'tempoEntrou':0, 
                                             'tempoSaiu':tempoEvento, 
                                             'tempoJogado':tempoEvento, 
                                             'situacao':'saiu',
                                             'equipe':equipe,
                                             'substituido':el[3].split(' - ')[-1],
                                             'substituiu':''})

                else:
                    jogador['tempoSaiu'] = tempoEvento
                    jogador['tempoJogado'] = jogador['tempoJogado'] - (tempo_1turno + tempo_2turno - tempoEvento)
                    jogador['situacao'] = 'entrou e saiu'

            # Jogadores
            global linhas
            self.jogadores = []

            in_ = dados.index(['3.0 - RELAÇÃO DE JOGADORES'])
            out = dados.index(['4.0 - COMISSÃO TÉCNICA'])

            linhas = [(el[:4], el[4:]) for el in dados[in_+3:out-2]]

            # Mandante
            global goleiros
            goleiros = []

            for index in [0,1]:
                goleiros.append(linhas[0][index][1])

            for goleiro in goleiros:
                if goleiro in [el['jogador'] for el in self.substitutos]:
                    goleiros.append(self.substitutos[goleiro]['substituido'])

    #         if len(goleiros)>2: print(index)

            for index in [0,1]:
                for linha in [el[index] for el in linhas if el[index] != ['-', '-', '-', '-']]:
                    numero = linha[0]
                    jogador = linha[1]
                    statusInicial = {'T':'titular','R':'reserva'}[linha[2]]
                    equipe = times_raw[index]
                    resultado = self.__dict__[['mandante','visitante'][index]]['resultado']
                    adversario = self.__dict__[['mandante','visitante'][[1,0][index]]]['equipe']

                    gols = []
                    for el in [el for el in self.gols if el['jogador']==jogador]:
                        gols.append(el['tempo'])

                    cartoesVermelhos = []
                    for el in [el for el in self.cartoesVermelhos if el['jogador']==jogador]:
                        cartoesVermelhos.append(el['tempo'])

                    cartoesAmarelos = []
                    for el in [el for el in self.cartoesAmarelos if el['jogador']==jogador]:
                        cartoesAmarelos.append(el['tempo'])

                    if jogador in [el['jogador'] for el in self.substitutos]:
                        data, = [el for el in self.substitutos if el['jogador']==jogador]
                        if data['equipe']==equipe:
                            tempoJogado = data['tempoJogado']
                            tempoEntrou = data['tempoEntrou']
                            tempoSaiu = data['tempoSaiu']
                            statusFinal = data['situacao']

                    else:
                        if statusInicial == 'titular':
                            tempoJogado = tempo_1turno + tempo_2turno
                            tempoEntrou = 0
                            tempoSaiu = tempoJogado
                            statusFinal = 'jogouTudo'

                        elif statusInicial == 'reserva':
                            tempoJogado = 0
                            tempoEntrou = 0
                            tempoSaiu = 0
                            statusFinal = 'naoJogou'


                    golsSofridos = []        
                    if jogador in goleiros:
                        for gol in [el for el in self.gols if el['equipe']==adversario]:
                            if (gol['tempo']<=tempoSaiu+4) & (gol['tempo']>=tempoEntrou):
                                golsSofridos.append(gol['tempo'])

                    self.jogadores.append({'jogador':jogador, 'numero':numero, 'statusInicial':statusInicial,
                                           'equipe':equipe, 'resultado':resultado, 'adversario':adversario,
                                           'gols':len(gols), 'golsTempo':l_(gols), 'golsSofridos':len(golsSofridos),
                                           'golsSofridosTempo':l_(golsSofridos), 'cartoesAmarelos':len(cartoesAmarelos), 
                                           'cartoesAmarelosTempo':l_(cartoesAmarelos), 'cartoesVermelhos':len(cartoesVermelhos),
                                           'cartoesVermelhosTempo':l_(cartoesVermelhos),'tempoJogado':tempoJogado,
                                           'tempoEntrou':tempoEntrou, 'tempoSaiu':tempoSaiu, 'statusFinal':statusFinal,
                                           'resultado':resultado, 'idJogo':self.idjogo, 'arbitro':self.arbitro})
                    self.scrape=True
                
        else:
            global flag
            flag = False
            self.scrape=False

In [9]:
sh = connect()
wk = sh.worksheet_by_title('dataJson')

uniques = wk.get_as_df().idJogo.unique()
games = [el for el in range(1,91) if el not in uniques]

# games = range(1,21)

l = []                    

flag = True
i = 0

while flag:
    for game in games:
        print(games[i])
        self = Partida(games[i])
        if self.scrape:
            l.extend(self.jogadores)
            i+=1
        else:
            flag=False
            break
            
update = pd.DataFrame(l)
wk.append_table(values = update.values.tolist())

# wk.set_dataframe(update, 'a1', fit=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


IndexError: range object index out of range

In [12]:
# Tabulação normal
sh = connect()

df = sh.worksheet_by_title('dataJson').get_as_df()

tb = df.groupby(['jogador','equipe'])

tb1 = tb[['cartoesAmarelos', 'cartoesVermelhos', 'gols',
       'golsSofridos','tempoJogado', ]].sum()
tb2 = df[['jogador','equipe','statusInicial', 'statusFinal']].melt(id_vars=['jogador','equipe']).pivot_table(index=['jogador','equipe'], columns='value', aggfunc='count').fillna(0).astype(int)

out = pd.concat([tb1, tb2['variable']], axis=1)

sh.worksheet_by_title('tabulação').set_dataframe(out.reset_index(), start='A1', fit=True)

# Tabulação dados marcados

tab = ['gols', 'cartoesAmarelos', 'cartoesVermelhos']
for column in tab:
    l_ = df[df[column]==1].values.tolist()

    for gol in df[df[column]>1].values:
        tempos = gol[9].split(', ')
        for tempo in tempos:
            l = [el for el in gol]
            l[9] = tempo
            l_.append(l)
            
    df_data = pd.DataFrame(l_)
    df_data.columns = df.columns
    search = re.compile(column).search
    df_data = df_data[['jogador','equipe','adversario','idJogo','resultado','statusInicial','statusFinal','arbitro']+[el for el in df_data.columns if search(el)]]
    df_data['faixaTempo'] = pd.cut(pd.to_numeric(df_data[column+'Tempo'], errors='coerce'), bins = [0,15,30,45,60,75,90,200]).astype('str')
    sh.worksheet_by_title(column).set_dataframe(df_data, start='A1', fit=True)
    
# Tabulação arbitros
tb = pd.concat([sh.worksheet_by_title(el).get_as_df() for el in ['cartoesAmarelos', 'cartoesVermelhos']], sort=False)
tb1 = tb.groupby('arbitro')[['cartoesAmarelos','cartoesVermelhos']].count()
tb2 = tb[['arbitro','idJogo']].drop_duplicates().groupby('arbitro').idJogo.count()

out = (tb1.T/tb2).T

sh.worksheet_by_title('arbitros').set_dataframe(out, start='A2', fit=True, copy_index=True)

# Cruzamentos

In [13]:
# # Existe uma "curva de tempo" dos gols e cartões? E substituições?
# sh = connect()
# sh.worksheets()

In [14]:
# for dado in ['gols','cartoesVermelhos','cartoesAmarelos']:
#     df = sh.worksheet_by_title(dado).get_as_df()
#     tb = df.pivot_table(dado,'equipe','faixaTempo','count', margins=True).astype('str').replace('nan','').replace('.0','', regex=True)
#     display(Markdown('# {}'.format(dado)))
#     display(tb)